### Create ImplementationGuide Resource from an excel based configuration file
python 3.7+

- Load Excel file using Pandas and access as dict
- Assign appropriate fields to IG resource ( including defaults )
- Add resources and pages based on the files in the target ig folders
- Validate resource
- Write resource to target ig folder

### Imports and globals

In [ ]:
import os #os module imported here
from json import load, dumps, loads
from requests import get, post, put
from IPython.display import display, Markdown, HTML
from pathlib import Path
from pandas import *
from datetime import datetime, date
from fhirclient.r4models import implementationguide as IG
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
import fhirclient.r4models.fhirreference as FR

#Globals

headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

### Get definition spreadsheet from folder


- right this is hard coded location - make a user input
- import data from spreadsheet
- convert to pandas df and then dict

In [ ]:
def_path=''

xls = ExcelFile(f'{def_path}test_ig_definition.xlsx')
df = read_excel(xls,sheet1,na_false = False)

df

In [ ]:
d = dict(zip(df.Element, df.Value))

### Assign appropriate fields to IG resource ( including defaults )
- Initiate ig resource instance
- assign id and defaults

In [ ]:
ig = IG.ImplementationGuide()
ig.id = f'{d["canonical"]}-{d["version"]}'

### Add resources and pages based on the files in the target ig folders

- for all files in folders inspect and add
- including spreadsheets using lmxml to get data

### Validate resource using the $validate operation

- using UHN R4 reference server

### Write resource to target ig folder

- use Pathlib here